In [95]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import time
import pandas as pd
import geopandas as gpd

In [96]:
# Chrome options to fix (unknown error: DevToolsActivePort file doesn't exist)
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [97]:
def get_job_info(location, role, num_jobs):
    
    # Initialize variables
    driver = webdriver.Chrome()
    driver.set_window_size(1120, 1000)
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + role + '"&locKeyword=United%20States&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=false&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []
    cities_states = []
    salary_range = []
    n = 1
    
    # Click the drop down menu for Date Posted
    date_posted_button = driver.find_element_by_xpath('//*[@id="filter_fromAge"]')
    date_posted_button.click()
    
    # Select Date Posted = 2 weeks
    date_range_object = driver.find_element_by_xpath('//*[@id="PrimaryDropdown"]/ul/li[5]')
    date_range_object.click()
    
    # Determine number of postings on page:
    count = 1
    while True:
        try:
            driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{count}]")
            count += 1
        except:
            break
    
    # While n < the number of postings on page
    while n < (count):
        # Click on a job posting
        time.sleep(1)
        job_button = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{n}]")
        job_button.click()
        time.sleep(0.5)
        
        # Attempt to close Sign Up Pop Up if any
        try:
            driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            print('closed sign up page')
        except:
            pass
        
        # Gather necessary information
        try:
            city_state_tag = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{n}]/div[2]/div[2]/span")
            salary_range_tag = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{n}]/div[2]/div[3]/div[1]/span")
            job_description_tag = driver.find_element_by_xpath("//*[@id='JobDescriptionContainer']")
        except:
            pass

        # DEBUG -- Save Info to List of Dictionaries
        try:
            #print(city_state_tag.text)
            #print(salary_range_tag.text)
            #print(job_description_tag.text)

            html = job_description_tag.get_attribute('innerHTML')
            soup = BeautifulSoup(html)
            job_description = soup.find('div', class_='jobDescriptionContent desc').text
            
            jobs.append(
                {
                    "job_title": role,
                    "location": city_state_tag.text,
                    "salary_range": salary_range_tag.text,
                    "job_description": job_description
                }
            )
        except:
            pass
        
        n += 1
    
    return jobs

In [90]:
jobs = get_job_info('United States', "data scientist", 5)

closed sign up page
closed sign up page


In [98]:
len(jobs)

24

In [99]:
jobs[1]

{'job_title': 'data scientist',
 'location': 'Washington, DC',
 'salary_range': '$74K - $128K (Glassdoor est.)',
 'job_description': "Do you want to build your brand by working for a leading consulting firm that drives eminence in the marketplace? Are you interested in leveraging your skills and strategic ideas to improve mission execution? If so, Deloitte could be the place for you! Our GPS Strategy and Analytics team brings deep industry expertise, rigorous analytical capabilities and a pragmatic mindset to help solve our client's most complex business problems. Join our team, and play a key role in helping to design our clients' roadmap to the future and help transform the marketplace.Work you'll doThe Data Scientist will join a team responsible for developing advanced analytics products; applying data visualization and statistical programming tools to enterprise data to advance and enable the key mission outcomes. In this role, she will support all phases of analytic work product d

In [100]:
# Results of scraping 1st page of GlassDoor
pd.DataFrame(jobs)

,job_title,location,salary_range,job_description
0,data scientist,"Herndon, VA",$79K - $131K (Glassdoor est.),Job Description:ATCS is currently seeking a qu...
1,data scientist,"Washington, DC",$74K - $128K (Glassdoor est.),Do you want to build your brand by working for...
2,data scientist,"Washington, DC",$86K - $143K (Glassdoor est.),Overview:Join our family of companies and let ...
3,data scientist,"Atlanta, GA",$67K - $113K (Glassdoor est.),Job BriefThis individual contributor role will...
4,data scientist,"Plano, TX",$67K - $110K (Glassdoor est.),"Please note, this role is not able to offer vi..."
5,data scientist,"Rockville, MD",$77K - $133K (Glassdoor est.),IAI is looking for a talented research scienti...
6,data scientist,"Portland, ME",$65K - $105K (Glassdoor est.),Job Posting End Date: April 20Unum is a compan...
7,data scientist,"Chantilly, VA",$80K - $165K (Glassdoor est.),Technology is constantly changing and our adve...
8,data scientist,"Milwaukee, WI",$71K - $126K (Glassdoor est.),What you will doThe Data Scientist will play a...
9,data scientist,"Winter Haven, FL",$52K - $117K (Glassdoor est.),"Neelands is a family owned, private Canadian c..."
